In [1]:
import os
import numpy as np
import glob
from tqdm import tqdm
import pandas as pd

from managedata.videos import Video
from managedata.extras import load_trials_descriptor


In [2]:
# base folder
folder_data = '/home/anaflo/MDMC/thesis/sensorium/data/'

# results folder
folder_results = os.path.join('..','intermediate_results')

# mice/recording folders
all_recording_folders = [
    'dynamic29156-11-10-Video-8744edeac3b4d1ce16b680916b5267ce',
    'dynamic29228-2-10-Video-8744edeac3b4d1ce16b680916b5267ce',
    'dynamic29234-6-9-Video-8744edeac3b4d1ce16b680916b5267ce',
    'dynamic29513-3-5-Video-8744edeac3b4d1ce16b680916b5267ce',
    'dynamic29514-2-9-Video-8744edeac3b4d1ce16b680916b5267ce',
]
    
# run for all recordings
for recording_folder in all_recording_folders:
    
    path_to_recording = os.path.join(folder_data, recording_folder)
    path_to_videos = os.path.join(folder_data, recording_folder,'data','videos')
    path_to_video_trials = glob.glob(os.path.join(path_to_videos,'*.npy'))
    print('\n=========================================================')
    print(f"Recording {recording_folder[0:18]} - {len(path_to_video_trials)} video files found\n")

    # create a folder for the outputs
    path_to_results = os.path.join(folder_results, recording_folder)
    if not os.path.exists(path_to_results):
        os.makedirs(path_to_results)
    path_to_results_metavideos = os.path.join(path_to_results, "meta_videos")
    if not os.path.exists(path_to_results_metavideos):
        os.makedirs(path_to_results_metavideos)

    
    # load the trials descriptor
    trial_type = load_trials_descriptor(path_to_recording, verbose=False)
    
    # compute for each video (trial)
    trial = []
    first_label = []
    label = []
    valid_frames = []
    n_peaks = []
    n_segments_peaks = []
    n_segments = []
    n_segment_bad = []
    avg_segment_duration = []
    for video_trial in tqdm(path_to_video_trials):

        # initialize class and load video
        video = Video(path_to_recording, video_trial)

        # run all the classification
        labels, segments = video.run_all()

        first_label_i = labels[0]
        n_segments_peaks_i = len(segments[1]["duration"])

        # save some metadata for each video to avoid recomputing later
        video.save_metadata(path_to_results_metavideos, metadata_for='examplar')
        
        # store to create a summary dataframe
        trial.append(video.trial)
        first_label.append(first_label_i)
        label.append(video.label)
        valid_frames.append(video.valid_frames)
        n_peaks.append(video.n_peaks)
        n_segments_peaks.append(n_segments_peaks_i)
        n_segments.append(video.n_segments)
        n_segment_bad.append(np.sum(video.segments["bad_properties"]))
        avg_segment_duration.append(np.mean(video.segments['duration']))

        
    # create a summary dataframe for all videos
    videos_df = pd.DataFrame({
                    'trial':trial, 
                    'trial_type': trial_type,
                    'valid_frames':valid_frames,
                    'first_label':first_label,
                    'label':label,
                    'peaks_n':n_peaks,
                    'segments_n_peaks':n_segments_peaks,
                    'segments_n':n_segments,
                    'segments_bad_n':n_segment_bad,
                    'segments_avg_duration':avg_segment_duration,
                    })
    
    # save the summary dataframe
    videos_df.to_csv(os.path.join(path_to_results,"videos_classification_table.csv"))


Recording dynamic29156-11-10 - 720 video files found



  0%|          | 0/720 [00:00<?, ?it/s]

 20%|██        | 145/720 [00:23<01:32,  6.20it/s]


KeyboardInterrupt: 